---

# Training Multilayer perception (Geotag - Latent space mapping)

---

In [18]:
import os
import pandas as pd
import numpy as np
import librosa
import tensorflow as tf
from tensorflow.keras import layers, models
import IPython.display as ipd
from tensorflow.keras.models import load_model

## Dataset

### Load audio file path-geotag mappings

In [3]:
df = pd.read_csv('../corpus/mappings.csv')

In [4]:
df.head()

,path,lat,long
0,corpus/audio/169884.mp3,0.501047,0.041804
1,corpus/audio/169885.mp3,0.501052,0.041806
2,corpus/audio/697381.mp3,0.187898,0.534158
3,corpus/audio/187893.mp3,0.459852,0.012109
4,corpus/audio/788102.mp3,0.236748,-0.876991


### Load audio files
- Crop audio to 44100 samples.
- Pad audio to 44100 samples.

In [9]:
audio = list()

for i, row in df.iterrows():
    y, sr = librosa.load(row['path'])

    y = np.pad(y, (0, max(0, 44100 - len(y))), mode="constant")[:44100]

    audio.append(y)

In [10]:
y = np.array(audio)
y.shape

(554, 44100)

### Get input vectors

In [20]:
X = df[['lat', 'long']].to_numpy()

## Get geotag-latent mapping

In [11]:
encoder = load_model('../models/encoder.h5')

In [12]:
latents = encoder.predict(y)

18/18 [==============================] - 5s 252ms/step


## Multilayer perceptron

In [66]:
mlp = tf.keras.Sequential()
mlp.add(layers.Input(shape=(2,)))
mlp.add(layers.Dense(256, activation='tanh'))
mlp.add(layers.Dense(128, activation='tanh'))
mlp.add(layers.Dense(64, activation='tanh'))

In [67]:
mlp.summary()

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_22 (Dense)            (None, 256)               768       
                                                                 
 dense_23 (Dense)            (None, 128)               32896     
                                                                 
 dense_24 (Dense)            (None, 64)                8256      
                                                                 
Total params: 41,920
Trainable params: 41,920
Non-trainable params: 0
_________________________________________________________________


In [68]:
mlp.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-1), loss='mean_squared_error')

In [69]:
mlp.fit(X, latents, epochs=50, validation_split=0.2, verbose=1)

Epoch 1/50
14/14 [==============================] - 0s 7ms/step - loss: 1.0301 - val_loss: 1.2777
Epoch 2/50
14/14 [==============================] - 0s 3ms/step - loss: 1.1283 - val_loss: 1.2679
Epoch 3/50
14/14 [==============================] - 0s 3ms/step - loss: 1.1414 - val_loss: 1.2825
Epoch 4/50
14/14 [==============================] - 0s 3ms/step - loss: 1.1434 - val_loss: 1.2825
Epoch 5/50
14/14 [==============================] - 0s 3ms/step - loss: 1.1434 - val_loss: 1.2825
Epoch 6/50
14/14 [==============================] - 0s 3ms/step - loss: 1.1434 - val_loss: 1.2825
Epoch 7/50
14/14 [==============================] - 0s 3ms/step - loss: 1.1434 - val_loss: 1.2825
Epoch 8/50
14/14 [==============================] - 0s 3ms/step - loss: 1.1434 - val_loss: 1.2825
Epoch 9/50
14/14 [==============================] - 0s 3ms/step - loss: 1.1434 - val_loss: 1.2825
Epoch 10/50
14/14 [==============================] - 0s 3ms/step - loss: 1.1434 - val_loss: 1.2825
Epoch 11/50
14/14 [

## Test

In [63]:
decoder = load_model('../models/decoder.h5')

In [70]:
example_geotag = np.array([X[0]])

In [71]:
latent = mlp.predict(example_geotag)
audio = decoder.predict(latent)

1/1 [==============================] - 0s 15ms/step


In [72]:
ipd.display(ipd.Audio(data=audio, rate=44100))

In [73]:
mlp.save('../models/mlp.h5')